In [25]:
from datetime import datetime, timedelta
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from oauth2client.service_account import ServiceAccountCredentials
from google.auth.transport.requests import Request

SCOPES = ['https://www.googleapis.com/auth/calendar.events']
SERVICE_ACCOUNT_KEY = 'PLACEHOLDER.json'
CALENDAR_ID = 'PLACEHOLDER@group.calendar.google.com'

COLOR_IDS = {
    'lodge': 11,
    'gh': 10,
    'fh': 9
}

In [26]:
# Create a service account in GCP, share the calendar with the corresponding email address.
# Setup credentials
creds = ServiceAccountCredentials.from_json_keyfile_name(
  SERVICE_ACCOUNT_KEY,
  scopes=SCOPES)

service = build('calendar', 'v3', credentials=creds)

In [27]:
# List 10 Events
now = datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time
print('Getting the upcoming 10 events')
events_result = service.events().list(calendarId=CALENDAR_ID, timeMin=now,
                                    maxResults=10, singleEvents=True,
                                    orderBy='startTime').execute()
events = events_result.get('items', [])

if not events:
    print('No upcoming events found.')
for event in events:
    start = event['start'].get('dateTime', event['start'].get('date'))
    print(start, event['summary'])

Getting the upcoming 10 events
2020-11-28T12:41:28-05:00 Google I/O 2015
2020-11-28T16:00:00-05:00 Name 1 & Name 2 - LODGE
2020-11-28T16:00:00-05:00 Name 1 & Name 2 - Inn
2020-11-28T16:00:00-05:00 Name 1 & Name 2 - GH


In [28]:
# Create new event
now = datetime.utcnow()
start = {"dateTime" : (now + timedelta(hours=1)).isoformat() + 'Z'}
end = {"dateTime" : (now + timedelta(hours=5)).isoformat() + 'Z'}

event = {
  'summary': 'Google I/O 2015',
  'location': '800 Howard St., San Francisco, CA 94103',
  'description': 'A chance to hear more about Google\'s developer products.',
  'colorId': COLOR_IDS['gh'],
  'start': start,
  'end': end,
}

event = service.events().insert(calendarId=CALENDAR_ID, body=event).execute()
print('Event created: ' + event.get('id'))

Event created: 3vvbfluhje6iug9tj3rttlcsbg
